In [34]:
from __future__ import print_function
import  requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import brown
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Debodeep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'brown'

In [2]:
movies_url="https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies3.html"

movies_response = requests.get(movies_url)

In [3]:
soup = BeautifulSoup(movies_response.text, 'html.parser')

In [4]:
movies_soup = BeautifulSoup(movies_response.text, 'html.parser')

In [5]:
links=movies_soup.findAll("a")

In [6]:
movies3=[]
for tag in links:
    page=tag.get('href')
    movies3.append(page)


In [7]:
len(movies3)

10000

In [8]:
from urllib.error import URLError,HTTPError,ContentTooShortError
import time


In [73]:
#j=1
#for i in range(len(movies3)):
    try:
        mv3ur1=requests.get(movies3[i])
        #soup = BeautifulSoup(mv3ur1.content, 'html.parser')
        #soup=soup.prettify("utf-8")
        strg="M3article_"+str(j)+'.html'
        j=j+1
        html_file=open(strg,"w",encoding='utf-8')
        html_file.write(mv3ur1.text)
        html_file.close()
        time.sleep(1)
    except(URLError,HTTPError,ContentTooShortError) as e:
        html=None
        time.sleep(20*60)
    
    


In [9]:
for i in range(1,len(movies3)): 
    path="C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw3\movies3\M3article_"+str(i)+".html"
    with open(path,"r",encoding="utf-8") as f:
      contents=f.read()
      page=BeautifulSoup(contents,'lxml') 

#### Function for page title

In [10]:
def title(page):
  return(page.find('h1',class_="firstHeading").get_text('i'))

#### Function for intro

In [11]:
def intro(page):
  content=page.find('div',class_="toc")
  if content != None:
    thres=content.findPreviousSiblings()
    intr_pg=''
    for i in thres:
      if i.name=="p":
        intr_pg+=i.text
      else: break
    return intr_pg
  elif content != None:
    thres=content.findNextSiblings()
    intr_pg=''
    for i in thres:
      if i.name=="p":
        intr_pg+=i.text
      else: break
  else: intr_pg="Null"
  return intr_pg

#### Function for Plot

In [12]:
def plot(page):
  for i in page.findAll(class_='mw-headline'):
    if i.text=='Plot':
      plt_hd=i.find_parent()
      plt_pg=""
      for i in plt_hd.find_next_siblings():
        if i.name=="p":
          plt_pg+=i.text
        else: break
    else: plt_pg="Null"
    return(plt_pg)

#### Function for infobox

In [13]:
def infobox(page):
    
    film_name = 'NA'
    director = "NA"
    producer = "NA"
    writer = "NA"
    starring = "NA"
    music = "NA"
    release_date = "NA"
    runtime = "NA"
    country = "NA"
    language = "NA"
    budget = "NA"
    for i in page.find_all('tr'):
            if page.find('th',{'class': ['summary']})!= None:
                    film_name = page.find('th',{'class': ['summary']} ).text.strip()
            if i.th:
                if(i.th.text.strip() == 'Directed by'):
                    if i.td:
                        director = i.td.text.strip()
                elif(i.th.text.strip() == 'Produced by'):
                    if i.td:
                        producer = i.td.text.strip()
                elif(i.th.text.strip() == 'Written by'):
                    if i.td:
                        writer = i.td.text.strip()
                elif(i.th.text.strip() == 'Starring'):
                    if i.td:
                        starring = i.td.text.strip()
                elif(i.th.text.strip() == 'Music by'):
                    if i.td:
                        music = i.td.text.strip()               
                elif(i.th.text.strip() == 'Release date'):
                    if i.td:
                        release_date = i.td.text.strip()
                elif(i.th.text.strip() == 'Running time'):
                    if i.td:
                        runtime = i.td.text.strip()
                elif(i.th.text.strip() == 'Country'):
                    if i.td:
                        country = i.td.text.strip()
                elif(i.th.text.strip() == 'Language'):
                      language = i.td.text.strip()
                elif(i.th.text.strip() == 'Budget'):
                      budget = i.td.text.strip()
    return(film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget)

In [24]:
import io
for i in range(1,len(movies3)+1):
    tsvname = 'Mv3tsv_tpi_'+str(i)+'.tsv'
    with io.open(tsvname,"w", encoding="utf-8") as f1: 
            path="C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw3\movies3\M3article_"+str(i)+".html"
            with open(path,"r",encoding="utf-8") as f:
                  contents=f.read()
                  page=BeautifulSoup(contents,'lxml') 
                  tsv_writer = csv.writer(f1, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow([title(page), intro(page),plot(page)]+list(infobox(page)))

#### A total picture

In [33]:
import io
with io.open('tsv_M3_total.tsv',"w", encoding="utf-8") as f1:
        for i in range(1,len(movies3)): 
            path="C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw3\movies3\M3article_"+str(i)+".html"
            with open(path,"r",encoding="utf-8") as f:
                contents=f.read()
                page=BeautifulSoup(contents,'lxml')  
                tsv_writer = csv.writer(f1, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring','music','release_date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow([title(page), intro(page),plot(page)]+list(infobox(page)))

In [26]:
with open('tsv_sample.tsv','wt',encoding="utf-8") as f1:
    headers=['title','intro', 'plot', 'film_name', 'director', 
                               'producer','writer', 'starring', 'music', 'release date', 
                               'runtime', 'country', 'language', 'budget']
    for i in range(1,20): 
        path=path="C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw3\movies3\M3article_"+str(i)+".html"
        with open(path,"r",encoding="utf-8") as f:
            contents=f.read()
            page=BeautifulSoup(contents,'lxml')  
            row= [title(page), intro(page),plot(page)]+list(infobox(page))
        writer = csv.writer(f1, delimiter='\t')
        writer.writerow(row)

In [40]:
dataset1 = pd.read_csv('tsv_M3_total.tsv', delimiter='\t')
dataset1

,title,intro,plot,film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget
0,Z.P.G.,"Z.P.G. (short for ""Zero Population Growth"") is...","Set in the future, the Earth has become severe...",Z.P.G.,Michael Campus,Frank De FelittaMax Simon EhrlichTom Madigan,Frank De FelittaMax Simon Ehrlich,Oliver ReedGeraldine ChaplinDon GordonDiane Ci...,Jonathan Hodge,25 May 1972 (1972-05-25),97 minutes,DenmarkUnited States[1],English,NaN
1,title,intro,plot,film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget
2,Zee and Co.,"The theme song ""Going in Circles"" was covered ...",Null,Zee and Co.,Brian G. Hutton,"Elliot KastnerJay KanterAlan Ladd, Jr.",Edna O'Brien,Elizabeth TaylorMichael CaineSusannah York,Stanley Myers,"January 21, 1972",110 minutes,United Kingdom,English,NaN
3,title,intro,plot,film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget
4,And Now the Screaming Starts!,"The screenplay, written by Roger Marshall, is ...",Null,And Now the Screaming Starts!,Roy Ward Baker,Max RosenbergMilton SubotskyGustave Berne,NaN,Peter CushingHerbert LomPatrick MageeIan Ogilv...,Douglas Gamley,"April 27, 1973",91 minutes,United Kingdom,English,"$500,000[1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19992,Jayami (2003 film),Jayam (English: Victory) is a 2003 Indian Tami...,The story is centered in a village called Push...,Jayam,M. Raja,M. VaralakshmiEditor Mohan,Prasanna Kumar (dialogues),RaviSadhaGopichand,R. P. Patnaik,20 June 2003 (2003-06-20),NaN,India,Tamil,NaN
19993,title,intro,plot,film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget
19994,Kaadhal Kondein,The story explores the mind of a youth who is ...,"Vinod (Dhanush), who has grown up under the ca...",Kaadhal Kondein,Selvaraghavan,Dr. K. Vimalageetha,Selvaraghavan,DhanushSonia AgarwalNageshSudeep SarangiDaniel...,Yuvan Shankar Raja,4 July 2003,185 minutes,India,Tamil,NaN
19995,title,intro,plot,film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget


#### Preprocessing text

In [ ]:
corpus1 = []
for i in range(len(dataset1)):
    plot = re.sub('[^a-zA-Z]', ' ',  str(dataset1['plot'][i]))
    plot = plot.lower()
    plot = plot.split()
    ps = PorterStemmer()
    plot = [ps.stem(word) for word in plot if not word in set(stopwords.words('english'))]
    plot = ' '.join(plot)
    corpus1.append(plot)

In [79]:
corpus1

['set futur earth becom sever pollut peopl need wear breath mask outsid sever overpopul affect avail resourc perman thick smog settl dismal citi cover earth entir surfac anim even common household pet extinct peopl eat tasteless bright color past plastic contain reduc world popul world govern decre children may conceiv next year break law result death penalti parent well newborn brainwash robot substitut use end yearn children death penalti ultim deterr place plastic dome suffoc death coupl child bear age visit babyland given life size animatron children instead russ oliv reed carol mcneil geraldin chaplin work museum recreat life th centuri carol desper child conceiv avoid abort machin instal bathroom remain pregnant child birth coupl must shield babi discov carol decid break law babi must avoid pri eye big brother like govern also grow jealousi friend neighbor find coupl real child go street scream babi babi author show neighbour georg gordon edna borden dian cilento find babi initi 

In [38]:
documents = [corpus1.split(" ") for i in corpus1]

NameError: name 'corpus1' is not defined

In [74]:
corpus2 = []
for i in range(len(dataset1)):
    intro = re.sub('[^a-zA-Z]', ' ', str(dataset1['intro'][i]))
    intro = intro.lower()
    intro = intro.split()
    ps = PorterStemmer()
    intro = [ps.stem(word) for word in intro if not word in set(stopwords.words('english'))]
    intro = ' '.join(intro)
    corpus2.append(intro)

In [76]:
corpus2

['z p g short zero popul growth danish american dystopian scienc fiction film direct michael campu star oliv reed geraldin chaplin inspir non fiction best sell book popul bomb paul r ehrlich film concern overpopul futur earth whose world govern execut violat year ban children film denmark film almost entir set bound featur art direct design reflect bleak oppress futur',
 'intro',
 'theme song go circl cover three dog night album seven separ fool well b side singl famili man citat need zee co also known x zee zee compani british drama film direct brian g hutton star elizabeth taylor michael cain susannah york releas columbia pictur base upon novel edna brien screenplay concern middl age bicker coupl whose marriag last leg woman come',
 'intro',
 'screenplay written roger marshal base novella fengriffen david case larg gothic hous use film oakley court near bray villag four star hotel scream start british gothic horror film direct roy ward baker star peter cush herbert lom patrick mage s

In [108]:
corpus3 = []
for i in range(len(dataset1)):
    title = re.sub('[^a-zA-Z]', ' ',  str(dataset1['title'][i]))
    title = title.lower()
    title = title.split()
    ps = PorterStemmer()
    title = [ps.stem(word) for word in title if not word in set(stopwords.words('english'))]
    title = ' '.join(title)
    corpus3.append(title)
corpus3

['z p g',
 'titl',
 'zee co',
 'titl',
 'scream start',
 'titl',
 'asphyx',
 'titl',
 'assassini film',
 'titl',
 'blockhous',
 'titl',
 'carri girl',
 'titl',
 'charley one eye',
 'titl',
 'commut husband',
 'titl',
 'creep flesh',
 'titl',
 'dark placesi film',
 'titl',
 'day jackali film',
 'titl',
 'digbi biggest dog world',
 'titl',
 'doll housei losey film',
 'titl',
 'lie say someth',
 'titl',
 'look',
 'titl',
 'england made mei film',
 'titl',
 'father dear fatheri film',
 'titl',
 'gawain green knighti film',
 'titl',
 'ghost noonday sun',
 'titl',
 'hirel',
 'titl',
 'hitler last ten day',
 'titl',
 'holiday buse',
 'titl',
 'homecomingi film',
 'titl',
 'horror hospit',
 'titl',
 'hous nightmar park',
 'titl',
 'legend hell hous',
 'titl',
 'live let diei film',
 'titl',
 'love ban',
 'titl',
 'man topi film',
 'titl',
 'never mind qualiti feel widthi film',
 'titl',
 'night watchi film',
 'titl',
 'darlingi film',
 'titl',
 'lucki man',
 'titl',
 'hous vanish',
 'titl',
 '